<a href="https://colab.research.google.com/github/Heng1222/Ohsumed_classification/blob/main/Model/task2_MLM_train_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import re
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForMaskedLM,
    DataCollatorForLanguageModeling,
    Trainer,
    TrainingArguments
)
from google.colab import drive

drive.mount('/content/drive')
MODEL_SAVE_PATH = "/content/drive/MyDrive/roberta_ohsumed_mlm"

def fetch_and_parse_data(url):
    response = requests.get(url)
    response.encoding = 'utf-8'
    content = response.text
    pattern = r'(.*?)\.,"([\s\S]*?)",(C\d+)'
    matches = re.findall(pattern, content)
    data_list = []
    for m in matches:
        data_list.append({
            'title': m[0].strip(),
            'abstract': m[1].replace('\n', ' ').strip()
        })
    return pd.DataFrame(data_list)

url = "https://media.githubusercontent.com/media/Heng1222/Ohsumed_classification/refs/heads/main/classification_data/ohsumed_dataset.csv"
df_all = fetch_and_parse_data(url)

df_all['full_text'] = df_all['title'] + " " + df_all['abstract']

model_name = "roberta-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    return tokenizer(examples["full_text"], truncation=True, max_length=256)

mlm_dataset = Dataset.from_pandas(df_all[['full_text']]).map(tokenize_function, batched=True)

model = AutoModelForMaskedLM.from_pretrained(model_name)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)

training_args = TrainingArguments(
    output_dir="./temp_mlm_checkpoints",
    overwrite_output_dir=True,
    num_train_epochs=5,
    per_device_train_batch_size=16,
    save_steps=5000,
    save_total_limit=2,
    prediction_loss_only=True,
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=mlm_dataset,
)

trainer.train()
trainer.save_model(MODEL_SAVE_PATH)
tokenizer.save_pretrained(MODEL_SAVE_PATH)

Mounted at /content/drive
正在從 GitHub 下載並解析資料...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/49161 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Step,Training Loss
500,1.360800
1000,1.295700
1500,1.264200
2000,1.240000
2500,1.212800
3000,1.192300
3500,1.163800
4000,1.156300
4500,1.134600
5000,1.128500


MLM 實驗模型已成功備份至 Google Drive: /content/drive/MyDrive/roberta_ohsumed_mlm
